In [317]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn import metrics
%matplotlib inline

import warnings
def ignore_warn():
    pass
warnings.warn = ignore_warn

In [318]:
data = pd.read_csv('./House_Rent_Dataset.csv')
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


### Предварительная обработка данных
В первую очередь проверим есть ли пропущенные значения

In [319]:
data.isna().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

Пропущенных значений нет

In [320]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [321]:
# у столбца Posted On указан тип object, преобразуем его во время.
data['Posted On'] = pd.to_datetime(data['Posted On'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Posted On          4746 non-null   datetime64[ns]
 1   BHK                4746 non-null   int64         
 2   Rent               4746 non-null   int64         
 3   Size               4746 non-null   int64         
 4   Floor              4746 non-null   object        
 5   Area Type          4746 non-null   object        
 6   Area Locality      4746 non-null   object        
 7   City               4746 non-null   object        
 8   Furnishing Status  4746 non-null   object        
 9   Tenant Preferred   4746 non-null   object        
 10  Bathroom           4746 non-null   int64         
 11  Point of Contact   4746 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 445.1+ KB


In [322]:
# Посмотрим на количество уникальных значений в каждом из столбцов
[(data.columns[i], data.iloc[:, i].unique().shape[0]) for i in range(len(data.columns))]

[('Posted On', 81),
 ('BHK', 6),
 ('Rent', 243),
 ('Size', 615),
 ('Floor', 480),
 ('Area Type', 3),
 ('Area Locality', 2235),
 ('City', 6),
 ('Furnishing Status', 3),
 ('Tenant Preferred', 3),
 ('Bathroom', 8),
 ('Point of Contact', 3)]

В столбце `Area Locality`, где хранится информация о местоположении дома, слишком много уникальных значений.

In [323]:
data = data.drop(['Area Locality'], axis = 1)

In [324]:
# Так как Floor содержит два параметра: 
# 1. общее количество этажей в доме
# 2. этаж, на котором располагаются аппартаменты,
# то эти данные стоит разделить на отдельные составляющие
data["Total Floors"] = data["Floor"].apply(lambda floor:floor.split()[-1])
data["Floor"] = data["Floor"].apply(lambda floor:floor.split()[0])

# Upper относится к уровню Upper Basement, Lower - к уровню Lower Basement.
# оба этих термина относятся к подвальным помещениям, но они отличаются, поэтому объединять их нельзя
encodes = {"Floor":     {"Ground": 0, "Upper": -1, "Lower": -2},
           "Total Floors": {"Ground": 0}}

data = data.replace(encodes)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Total Floors
0,2022-05-18,2,10000,1100,0,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,2
1,2022-05-13,2,20000,800,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
2,2022-05-16,2,17000,1000,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
3,2022-07-04,2,10000,800,1,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2
4,2022-05-09,2,7500,850,1,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner,2


Поценциально цена аренды может зависить от даты, т.к. в зависимости от экономической ситуации и других факторов цена может меняться, но все данные в `Posted On` находятся в пределах 3 месяц одно года.

Данные о дате публикации объявления `Posted On` возможно разбить на составляющие: день, месяц, день недели и пр., но это тоже имеет мало смысла (и к тому же эмпирически было проверено, что это увеличивает значение функции потерь), поэтому избавимся от данных о дате публикации.

In [325]:
data = data.drop('Posted On', axis = 1)
data.head()

,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Total Floors
0,2,10000,1100,0,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,2
1,2,20000,800,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
2,2,17000,1000,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
3,2,10000,800,1,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2
4,2,7500,850,1,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner,2


In [326]:
columns = ['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']
data = pd.get_dummies(data, columns=columns)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   BHK                                4746 non-null   int64 
 1   Rent                               4746 non-null   int64 
 2   Size                               4746 non-null   int64 
 3   Floor                              4746 non-null   object
 4   Bathroom                           4746 non-null   int64 
 5   Total Floors                       4746 non-null   object
 6   Area Type_Built Area               4746 non-null   bool  
 7   Area Type_Carpet Area              4746 non-null   bool  
 8   Area Type_Super Area               4746 non-null   bool  
 9   City_Bangalore                     4746 non-null   bool  
 10  City_Chennai                       4746 non-null   bool  
 11  City_Delhi                         4746 non-null   bool  
 12  City_H

,BHK,Rent,Size,Floor,Bathroom,Total Floors,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,0,2,2,False,False,True,False,...,False,False,False,True,False,True,False,False,False,True
1,2,20000,800,1,1,3,False,False,True,False,...,False,False,True,False,False,True,False,False,False,True
2,2,17000,1000,1,1,3,False,False,True,False,...,False,False,True,False,False,True,False,False,False,True
3,2,10000,800,1,1,2,False,False,True,False,...,False,False,False,True,False,True,False,False,False,True
4,2,7500,850,1,1,2,False,True,False,False,...,False,False,False,True,True,False,False,False,False,True


In [327]:

data.corr()['Rent'].sort_values(ascending=False)

Rent                                 1.000000
Bathroom                             0.441215
Size                                 0.413551
BHK                                  0.369718
Total Floors                         0.352268
Point of Contact_Contact Agent       0.339750
City_Mumbai                          0.327038
Floor                                0.326200
Area Type_Carpet Area                0.215769
Furnishing Status_Furnished          0.110576
Tenant Preferred_Family              0.063941
Furnishing Status_Semi-Furnished     0.045309
Tenant Preferred_Bachelors           0.042151
Point of Contact_Contact Builder    -0.005482
Area Type_Built Area                -0.006439
City_Delhi                          -0.027072
City_Bangalore                      -0.061512
Tenant Preferred_Bachelors/Family   -0.078774
City_Chennai                        -0.082361
City_Hyderabad                      -0.087465
City_Kolkata                        -0.105322
Furnishing Status_Unfurnished     

### Разбиваем данные и масштабируем данные

In [328]:
# Разделение на признаки и целевую переменную
X = data.drop('Rent', axis=1)
y = data['Rent']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [329]:
# Нормализуем данные
scaler = StandardScaler()
sc_y = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Найдем решение задачи в замкнутом виде с помощью псевдообратной матрицы.
Для этого воспользуемся функциями из numpy, а позже получим те же значения с помощью функций из sklearn для сравнения

In [330]:
# Найдем решение с помощью псевдообратной матрицы для OLS
def closed_form_ols(X, y):
    X_transpose = np.transpose(X)
    X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X))
    X_transpose_y = np.dot(X_transpose, y)
    theta = np.dot(X_transpose_X_inv, X_transpose_y)
    return theta

theta_ols = closed_form_ols(X_train, y_train)
print(theta_ols)

[  2795.53852166  22460.9550461    5578.31202734   7919.51391263
   3281.93541702    225.74252273   9827.5092637    7278.65489055
   2499.52281849    363.72078277   4553.36387738   1119.54200854
   -269.82604279  20595.33673999   3321.6474623    -510.08090385
   -341.93765345   -134.06719326   -389.78255466  -3620.09941746
  -9544.18858646     69.11059122 -14938.96533395]


In [331]:
# Найдем решение с помощью псевдообратной матрицы для Ridge
def closed_form_ridge(X, y, alpha):
    X_transpose = np.transpose(X)
    identity = np.eye(X.shape[1])
    X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X) + alpha * identity)
    X_transpose_y = np.dot(X_transpose, y)
    theta = np.dot(X_transpose_X_inv, X_transpose_y)
    return theta

theta_ridge = closed_form_ridge(X_train, y_train, alpha=0.1)
print(theta_ridge)

[ 2722.20536134 23137.0440888   5752.49369348  8897.63682497
  2522.33203512   131.33462863   783.13411892  -789.13821637
 -1853.77743693 -4661.86872968  1164.64746114 -8287.57295912
 -2043.10849416 14848.02370706  2511.56924502 -1183.81510525
  -585.68625512  1123.35297224   905.56369145 -2793.03518383
  1552.74704629   484.56408634 -1569.24136337]


In [332]:
def gradient_descent(X, y, theta_init, learning_rate, num_iterations):
    theta = theta_init.copy()
    m = len(y)
    for iteration in range(num_iterations):
        error = np.dot(X, theta) - y
        gradient = (1/m) * np.dot(X.T, error)
        theta = theta - learning_rate * gradient  # Reshape the gradient array
    return theta

X_train_with_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_with_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

theta_init = np.zeros(X_train_with_bias.shape[1])  # Инициализируем нулями
learning_rate = 0.01 
num_iterations = 1000  

theta_gradient_descent = gradient_descent(X_train_with_bias, y_train, theta_init, learning_rate, num_iterations)

print(theta_gradient_descent)

[35149.99879817  3287.30127843 21948.81102828  5163.59323822
  9356.27238342  3217.2618016    136.80105257   744.81415653
  -751.07056067 -1696.47197865 -4565.18246621  1040.6926467
 -8116.26768625 -1976.21325183 14488.62999508  2566.72545424
 -1187.71032127  -621.24301889  1138.86545415   900.97124451
 -2805.98735904  1667.06168271   478.2682147  -1683.31281693]


In [333]:
#Посмотрим на полученные модели
n = X.shape[1] - 1
linear_function_ols = "y = "
for i in range(n):
    linear_function_ols += f"{theta_ols[i]} * x{i+1} + "
linear_function_ols += f"{theta_ols[n]}"
print("Линейная регрессия:", linear_function_ols)


linear_function_ridge = "y = "
for i in range(n):
    linear_function_ridge += f"{theta_ridge[i]} * x{i+1} + "
linear_function_ridge += f"{theta_ridge[n]}"
print("Ridge (L2):", linear_function_ridge)


linear_function_gradient_descent = "y = "
for i in range(n):
    linear_function_gradient_descent += f"{theta_gradient_descent[i]} * x{i+1} + "
linear_function_gradient_descent += f"{theta_gradient_descent[n]}"
print("Градиентный спуск:", linear_function_gradient_descent)


Линейная регрессия: y = 2795.5385216590703 * x1 + 22460.955046095994 * x2 + 5578.312027344423 * x3 + 7919.513912631834 * x4 + 3281.9354170197867 * x5 + 225.7425227272495 * x6 + 9827.509263701428 * x7 + 7278.654890551041 * x8 + 2499.5228184932394 * x9 + 363.7207827687662 * x10 + 4553.363877377284 * x11 + 1119.542008536865 * x12 + -269.826042794376 * x13 + 20595.336739991188 * x14 + 3321.647462297422 * x15 + -510.0809038461937 * x16 + -341.9376534543592 * x17 + -134.06719326475195 * x18 + -389.78255466431074 * x19 + -3620.0994174630505 * x20 + -9544.188586455217 * x21 + 69.11059121932703 * x22 + -14938.965333954442
Ridge (L2): y = 2722.205361341939 * x1 + 23137.044088801864 * x2 + 5752.493693475629 * x3 + 8897.636824971314 * x4 + 2522.3320351197526 * x5 + 131.3346286320296 * x6 + 783.1341189234336 * x7 + -789.1382163691924 * x8 + -1853.7774369345843 * x9 + -4661.868729677443 * x10 + 1164.647461136578 * x11 + -8287.572959115943 * x12 + -2043.1084941582826 * x13 + 14848.02370705638 * x14 +

In [334]:

def print_metrics_for_theta(X_test,theta):
    y_pred = np.dot(X_test, theta)
    mae = np.mean(np.abs(y_pred - y_test))
    mse = np.mean((y_pred - y_test) ** 2)
    rmse = np.sqrt(mse)
    print("Mean Absolute Error (MAE):", mae)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("\n")

print("Линейная регрессия")
print_metrics_for_theta(X_test, theta_ols)
print("Ridge (L2)")
print_metrics_for_theta(X_test, theta_ridge)
print("Градиентный спуск")
print_metrics_for_theta(X_test_with_bias, theta_gradient_descent)


Линейная регрессия
Mean Absolute Error (MAE): 37484.47502839073
Mean Squared Error (MSE): 3096275469.924941
Root Mean Squared Error (RMSE): 55644.18630840909


Ridge (L2)
Mean Absolute Error (MAE): 37060.76302232031
Mean Squared Error (MSE): 3071243471.5428176
Root Mean Squared Error (RMSE): 55418.800704659945


Градиентный спуск
Mean Absolute Error (MAE): 21689.1880015194
Mean Squared Error (MSE): 1917285475.2442055
Root Mean Squared Error (RMSE): 43786.81851018872




### Какая модель имеет наименьшее значение функции потерь на тестовой выборке?


**Наименьшее значение функции потерь MSE на тестовой выборке имеет модель, полученная градиентным спуском, следовательно, она имеет наилучшую точность**


Значение функции потерь для случая с L2 регуляцией меньше, чем без него. Регуляризация помогает избежать эффекта переобучения в данном примере. Использование L2-регуляризации (Ridge) позволяет штрафовать большие значения коэффициентов модели, что помогает уменьшить переобучение.

### Моделирование


In [335]:
from numpy import ndarray
# Модель линейной регрессии без регуляризации
ols_linear_reg = LinearRegression()
ols_linear_reg.fit(X_train, y_train)
ols_linear_reg_prediction = ols_linear_reg.predict(X_test)

def print_metrics(prediction: ndarray):
    # Evaluation metrics
    mae = metrics.mean_absolute_error(y_test, prediction)
    mse=  metrics.mean_squared_error(y_test, prediction)
    rmse =  np.sqrt(mse)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('\n')

print("Значения функции потерь (линейная регрессия):")
print_metrics(ols_linear_reg_prediction)


Значения функции потерь (линейная регрессия):
MAE: 21706.591252644037
MSE: 1908284470.1889894
RMSE: 43683.91546311971




In [336]:
# Модель линейной регрессии с L2-регуляризацией (Ridge)
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(X_train, y_train)
ridge_reg_prediction = ridge_reg.predict(X_test)
print("Значения функции потерь (регуляризация Ridge):")
print_metrics(ridge_reg_prediction)

Значения функции потерь (регуляризация Ridge):
MAE: 21775.902375557213
MSE: 1908288474.9553783
RMSE: 43683.96130109286




Получим модель линейной регрессии с помощью стохастического градиентного спуска

In [337]:
sgdr_reg = SGDRegressor(tol=.0001, eta0=.01) 
sgdr_reg.fit(X_train, y_train)
sgdr_reg_prediction = sgdr_reg.predict(X_test)
print("Значения функции потерь (Стохастический градиентный спуск):")
print_metrics(sgdr_reg_prediction)

n = X.shape[1] - 1

# Найдем градиент
theta = sgdr_reg.coef_ # коэффициенты
m = len(X_train)  # количество характеристик

gradient = (1/m) * np.dot(X_train.T, (np.dot(X_train, theta) - y_train))

print(f"Градиент:\n {gradient}")

Значения функции потерь (Стохастический градиентный спуск):
MAE: 70707.9044213136
MSE: 9958804295.727444
RMSE: 99793.80890479851


Градиент:
 [-6.87030103e+04 -5.44664826e+04 -3.41006936e+04 -6.40906830e+04
 -3.71649273e+04  2.43133746e+02 -5.62982923e+04  5.62852195e+04
 -2.17867247e+04 -1.85763426e+04 -1.08181961e+04  7.94714216e+04
 -4.31738827e+03 -2.41372351e+04 -1.60961170e+04 -5.29175073e+04
  6.58790348e+04 -2.72120263e+04  3.68117016e+04 -2.03637812e+04
 -3.55738681e+04  9.16320440e+05  3.74476785e+03]


### Какие признаки оказывают наибольший вклад в точность определения стоимости аренды?

Чтобы определить признаки, которые оказывают наибольший вклад в точность определения стоимости аренды, можно использовать коэффициенты модели линейной регрессии. Чем больше абсолютное значение коэффициента, тем больший вклад вносит соответствующий признак.

In [338]:
# Выведем все коэффициенты от самого высокого по модулю до самого низкого (т.е. от более значимых до менее значимых) для каждой из представленных моделей
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ols_linear_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для линейной регрессии:")
print(feature_importance_sorted)

Наиболее важные признаки для линейной регрессии:
                              Признак   Коэффициент
22     Point of Contact_Contact Owner  1.103032e+17
20     Point of Contact_Contact Agent  1.102796e+17
15   Furnishing Status_Semi-Furnished  6.026663e+16
16      Furnishing Status_Unfurnished  5.869397e+16
14        Furnishing Status_Furnished  4.209352e+16
18  Tenant Preferred_Bachelors/Family -2.582583e+16
13                        City_Mumbai  2.334541e+16
8                      City_Bangalore  2.282370e+16
9                        City_Chennai  2.269118e+16
7                Area Type_Super Area  2.243119e+16
6               Area Type_Carpet Area  2.243043e+16
11                     City_Hyderabad  2.226859e+16
17         Tenant Preferred_Bachelors -2.203757e+16
10                         City_Delhi  1.942067e+16
12                       City_Kolkata  1.791660e+16
19            Tenant Preferred_Family -1.723677e+16
21   Point of Contact_Contact Builder  3.830549e+15
5              

In [339]:

feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ridge_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для Ridge:")
print(feature_importance_sorted)

Наиболее важные признаки для Ridge:
                              Признак   Коэффициент
1                                Size  23130.631498
13                        City_Mumbai  14844.916728
3                            Bathroom   8898.653598
11                     City_Hyderabad  -8285.814975
2                               Floor   5751.147271
9                        City_Chennai  -4660.892209
19            Tenant Preferred_Family  -2792.663386
0                                 BHK   2724.868636
4                        Total Floors   2524.591151
14        Furnishing Status_Furnished   2511.774713
12                       City_Kolkata  -2042.747600
8                      City_Bangalore  -1852.993997
22     Point of Contact_Contact Owner  -1570.194887
20     Point of Contact_Contact Agent   1553.705472
15   Furnishing Status_Semi-Furnished  -1183.716772
10                         City_Delhi   1163.971919
17         Tenant Preferred_Bachelors   1123.179618
18  Tenant Preferred_Bachelo

In [340]:
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": sgdr_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для градиентного спуска:")
print(feature_importance_sorted)

Наиболее важные признаки для градиентного спуска:
                              Признак    Коэффициент
21   Point of Contact_Contact Builder  911914.737392
11                     City_Hyderabad   32798.358167
22     Point of Contact_Contact Owner  -30525.899121
16      Furnishing Status_Unfurnished   21777.522970
0                                 BHK  -18532.005259
15   Furnishing Status_Semi-Furnished  -18523.463249
7                Area Type_Super Area   14435.290455
6               Area Type_Carpet Area  -14385.924169
9                        City_Chennai  -14144.911137
8                      City_Bangalore  -13192.123025
12                       City_Kolkata   -9789.181378
1                                Size    9420.395380
18  Tenant Preferred_Bachelors/Family    9312.825493
13                        City_Mumbai    8679.013267
3                            Bathroom   -7790.393140
17         Tenant Preferred_Bachelors   -7446.036443
10                         City_Delhi   -6979.237